In [ ]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
import pandas as pd
from datetime import timedelta

# Load the data
FILE_PATH = 'HealthData.csv'
data = pd.read_csv(FILE_PATH)

def detect_epilepsy(health_data):
    """Detect potential epileptic episodes based on health metrics.
    
    Args:
        health_data: DataFrame containing health metrics with datetime index
        
    Returns:
        DataFrame with added epilepsy alert column
    """
    # Calculate rolling statistics for key metrics
    window_size = 5  # Number of consecutive readings to analyze
    health_data['heartrate_rolling_std'] = health_data['heartrate'].rolling(window=window_size).std()
    health_data['oxygen_rolling_std'] = health_data['blood_oxygen_level'].rolling(window=window_size).std()
    health_data['temp_rolling_std'] = health_data['body_temperature'].rolling(window=window_size).std()
    
    # Define thresholds for epilepsy detection
    hr_threshold = 30  # BPM standard deviation threshold
    oxygen_threshold = 3  # % oxygen standard deviation threshold
    temp_threshold = 0.5  # °C standard deviation threshold
    
    # Detect potential epileptic episodes
    health_data['epilepsy_alert'] = False
    
    for i in range(len(health_data)):
        abnormal_metrics = 0
        if health_data.loc[i, 'heartrate_rolling_std'] > hr_threshold:
            abnormal_metrics += 1
        if health_data.loc[i, 'oxygen_rolling_std'] > oxygen_threshold:
            abnormal_metrics += 1
        if health_data.loc[i, 'temp_rolling_std'] > temp_threshold:
            abnormal_metrics += 1
        
        if abnormal_metrics >= 2:
            health_data.loc[i, 'epilepsy_alert'] = True
    
    return health_data

def create_epilepsy_detection_figure(health_data):
    """Create enhanced visualization for epilepsy detection monitoring.
    
    Args:
        health_data: DataFrame containing health metrics with epilepsy alerts
        
    Returns:
        plotly Figure object
    """
    fig = go.Figure()
    
    # Add traces for the metrics with more visual distinction
    fig.add_trace(go.Scatter(
        x=health_data['datetime'], 
        y=health_data['heartrate'],
        name='Heart Rate (BPM)',
        line={"color": "#636EFA", "width": 2},
        yaxis='y1',
        mode='lines+markers',
        marker={"size": 4}
    ))
    
    fig.add_trace(go.Scatter(
        x=health_data['datetime'], 
        y=health_data['blood_oxygen_level'],
        name='Blood Oxygen (%)',
        yaxis='y2',
        line={"color": "#00CC96", "width": 2, "dash": "dot"},
        mode='lines+markers',
        marker={"size": 4}
    ))
    
    fig.add_trace(go.Scatter(
        x=health_data['datetime'], 
        y=health_data['body_temperature'],
        name='Body Temp (°C)',
        yaxis='y3',
        line={"color": "#EF553B", "width": 2, "dash": "dash"},
        mode='lines+markers',
        marker={"size": 4}
    ))
    
    # Highlight potential epilepsy events with more prominent markers
    epilepsy_events = health_data[health_data['epilepsy_alert']]
    if not epilepsy_events.empty:
        fig.add_trace(go.Scatter(
            x=epilepsy_events['datetime'],
            y=epilepsy_events['heartrate'],
            mode='markers',
            name='⚠️ Potential Epilepsy',
            marker={
                "color": "gold",
                "size": 16,
                "line": {"width": 2, "color": "red"},
                "symbol": "hexagram"
            },
            hovertemplate='<b>Potential Epilepsy Event</b><br>' +
                         'Time: %{x}<br>' +
                         'Heart Rate: %{y} BPM<extra></extra>'
        ))
    
    # Add shaded regions for epilepsy events
    for _, event in epilepsy_events.iterrows():
        fig.add_vrect(
            x0=event['datetime'] - timedelta(minutes=5),
            x1=event['datetime'] + timedelta(minutes=5),
            fillcolor="red",
            opacity=0.1,
            layer="below",
            line_width=0
        )
    
    # Update layout for better readability
    fig.update_layout(
        title={
            'text': '<b>Epilepsy Detection Monitoring</b>',
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 24}
        },
        xaxis={
            "title": "Time",
            "gridcolor": "lightgray",
            "showline": True,
            "linewidth": 1,
            "linecolor": "black"
        },
        yaxis={
            "title": {"text": "<b>Heart Rate (BPM)</b>", "font": {"color": "#636EFA"}},
            "side": "left",
            "position": 0.05,
            "gridcolor": "#f0f0f0",
            "showline": True,
            "linewidth": 1,
            "linecolor": "#636EFA"
        },
        yaxis2={
            "title": {"text": "<b>Blood Oxygen (%)</b>", "font": {"color": "#00CC96"}},
            "overlaying": "y",
            "side": "right",
            "position": 0.85,
            "gridcolor": "#f0f0f0",
            "showline": True,
            "linewidth": 1,
            "linecolor": "#00CC96"
        },
        yaxis3={
            "title": {"text": "<b>Body Temp (°C)</b>", "font": {"color": "#EF553B"}},
            "overlaying": "y",
            "side": "right",
            "gridcolor": "#f0f0f0",
            "showline": True,
            "linewidth": 1,
            "linecolor": "#EF553B"
        },
        legend={
            "orientation": "h",
            "yanchor": "bottom",
            "y": 1.02,
            "xanchor": "right",
            "x": 1,
            "bgcolor": "rgba(255,255,255,0.5)"
        },
        hovermode="x unified",
        plot_bgcolor="white",
        paper_bgcolor="white",
        margin={"t": 100, "b": 80, "l": 80, "r": 80},
        height=600
    )
    
    # Add annotations for epilepsy events
    if not epilepsy_events.empty:
        annotations = []
        for _, event in epilepsy_events.iterrows():
            annotations.append(dict(
                x=event['datetime'],
                y=event['heartrate'] + 10,
                xref="x",
                yref="y1",
                text="⚠️",
                showarrow=True,
                arrowhead=2,
                arrowsize=1,
                arrowwidth=2,
                arrowcolor="red",
                ax=0,
                ay=-30,
                font=dict(size=16, color="red")
            ))
        fig.update_layout(annotations=annotations)
    
    return fig

def create_oxygen_trend_figure(health_data, window_size=5, alert_threshold=92):
    """Create enhanced blood oxygen trend visualization with alerts.
    
    Args:
        health_data: DataFrame containing health metrics
        window_size: Size of rolling window for average calculation
        alert_threshold: Oxygen level threshold for alerts
        
    Returns:
        plotly Figure object
    """
    # Calculate rolling average and standard deviation
    health_data['rolling_oxygen'] = health_data['blood_oxygen_level'].rolling(window=window_size).mean()
    health_data['rolling_std'] = health_data['blood_oxygen_level'].rolling(window=window_size).std()
    
    # Create figure
    fig = go.Figure()
    
    # Add oxygen level trace with confidence band
    fig.add_trace(go.Scatter(
        x=health_data['datetime'],
        y=health_data['blood_oxygen_level'],
        mode='lines',
        name='Blood Oxygen Level',
        line={"color": "#1f77b4", "width": 2},
        hovertemplate='<b>Oxygen Level</b>: %{y}%<extra></extra>'
    ))
    
    # Add confidence band
    fig.add_trace(go.Scatter(
        x=health_data['datetime'],
        y=health_data['rolling_oxygen'] + health_data['rolling_std'],
        mode='lines',
        line={"width": 0},
        showlegend=False,
        hoverinfo='skip'
    ))
    
    fig.add_trace(go.Scatter(
        x=health_data['datetime'],
        y=health_data['rolling_oxygen'] - health_data['rolling_std'],
        mode='lines',
        line={"width": 0},
        fillcolor='rgba(31, 119, 180, 0.2)',
        fill='tonexty',
        showlegend=False,
        name='Standard Deviation',
        hovertemplate='<b>Oxygen Range</b>: %{y:.1f}%<extra></extra>'
    ))
    
    # Add rolling average trace
    fig.add_trace(go.Scatter(
        x=health_data['datetime'],
        y=health_data['rolling_oxygen'],
        mode='lines',
        name=f'Rolling Average ({window_size} points)',
        line={"color": "#ff7f0e", "width": 2, "dash": "dot"},
        hovertemplate='<b>Avg Oxygen</b>: %{y:.1f}%<extra></extra>'
    ))
    
    # Add alert threshold line
    fig.add_shape(
        type="line",
        x0=health_data['datetime'].min(),
        y0=alert_threshold,
        x1=health_data['datetime'].max(),
        y1=alert_threshold,
        line={"color": "red", "width": 3, "dash": "dash"},
        name="Alert Threshold",
        layer="below"
    )
    
    # Add threshold label
    fig.add_annotation(
        x=health_data['datetime'].max(),
        y=alert_threshold + 0.5,
        text=f"Alert Threshold: {alert_threshold}%",
        showarrow=False,
        xanchor="right",
        font={"color": "red", "size": 12}
    )
    
    # Highlight points below threshold with more prominent markers
    alert_points = health_data[health_data['blood_oxygen_level'] < alert_threshold]
    if not alert_points.empty:
        fig.add_trace(go.Scatter(
            x=alert_points['datetime'],
            y=alert_points['blood_oxygen_level'],
            mode='markers',
            name='Respiratory Alert',
            marker={
                "color": "red",
                "size": 10,
                "line": {"width": 2, "color": "white"},
                "symbol": "diamond"
            },
            hovertemplate='<b>Alert!</b> Oxygen: %{y}%<br>Time: %{x}<extra></extra>'
        ))
        
        # Add shaded regions for alert periods
        alert_groups = (alert_points['datetime'].diff() > timedelta(minutes=5)).cumsum()
        for _, group in alert_points.groupby(alert_groups):
            fig.add_vrect(
                x0=group['datetime'].min() - timedelta(minutes=2),
                x1=group['datetime'].max() + timedelta(minutes=2),
                fillcolor="red",
                opacity=0.1,
                layer="below",
                line_width=0
            )
    
    # Update layout for better readability
    fig.update_layout(
        title={
            'text': f'<b>Blood Oxygen Monitoring with Respiratory Alerts</b><br><sup>Threshold: {alert_threshold}% | Window Size: {window_size} points</sup>',
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 22}
        },
        xaxis={
            "title": "Time",
            "gridcolor": "lightgray",
            "showline": True,
            "linewidth": 1,
            "linecolor": "black"
        },
        yaxis={
            "title": "Blood Oxygen Level (%)",
            "range": [85, 100],
            "gridcolor": "#f0f0f0",
            "showline": True,
            "linewidth": 1,
            "linecolor": "black"
        },
        hovermode="x unified",
        legend={
            "orientation": "h",
            "yanchor": "bottom",
            "y": 1.02,
            "xanchor": "right",
            "x": 1,
            "bgcolor": "rgba(255,255,255,0.5)"
        },
        plot_bgcolor="white",
        paper_bgcolor="white",
        margin={"t": 120, "b": 80, "l": 80, "r": 80},
        height=600
    )
    
    # Add annotations for alerts
    if not alert_points.empty:
        # Group alerts that are close together
        alert_groups = (alert_points['datetime'].diff() > timedelta(minutes=5)).cumsum()
        
        annotations = []
        for _, group in alert_points.groupby(alert_groups):
            min_time = group['datetime'].min()
            min_oxygen = group['blood_oxygen_level'].min()
            
            annotations.append(dict(
                x=min_time,
                y=min_oxygen - 1,
                xref="x",
                yref="y",
                text=f"⚠️ Low Oxygen: {min_oxygen}%",
                showarrow=True,
                arrowhead=2,
                arrowsize=1,
                arrowwidth=2,
                arrowcolor="red",
                ax=0,
                ay=30,
                font=dict(size=12, color="red"),
                bordercolor="red",
                borderwidth=1,
                borderpad=4,
                bgcolor="white"
            ))
        
        fig.update_layout(annotations=annotations)
    
    return fig

# Preprocess the data
data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['timestamp'])
data_cleaned = data.copy()
for col in ['heartrate', 'blood_oxygen_level', 'body_temperature', 'air_quality', 'room_temperature', 'hydration_levels']:
    data_cleaned[col] = pd.to_numeric(data_cleaned[col], errors='coerce')
data_cleaned = data_cleaned.dropna()

# Calculate Heart Rate Variability (HRV)
data_cleaned['hrv'] = data_cleaned['heartrate'].diff()

# Correlation matrix
correlation_matrix = data_cleaned[['heartrate', 'blood_oxygen_level', 'body_temperature', 'air_quality', 'room_temperature', 'hydration_levels']].corr()

# Apply epilepsy detection
data_cleaned = detect_epilepsy(data_cleaned)
epilepsy_events = data_cleaned[data_cleaned['epilepsy_alert']]

# Summary Statistics
summary_stats = {
    'Average Heart Rate (BPM)': round(data_cleaned['heartrate'].mean(), 2),
    'Average Blood Oxygen Level (%)': round(data_cleaned['blood_oxygen_level'].mean(), 2),
    'Average Body Temperature (°C)': round(data_cleaned['body_temperature'].mean(), 2),
    'Average Room Temperature (°C)': round(data_cleaned['room_temperature'].mean(), 2),
    'Average Hydration Levels (%)': round(data_cleaned['hydration_levels'].mean(), 2),
    'Total Falls Detected': int(data_cleaned['fall_detection'].sum()),
    'Potential Epilepsy Events': int(data_cleaned['epilepsy_alert'].sum()),
    'Overall Health Score': round(((data_cleaned['heartrate'].mean() * 0.3) +
                                (data_cleaned['blood_oxygen_level'].mean() * 0.3) +
                                (data_cleaned['body_temperature'].mean() * 0.2) +
                                (data_cleaned['hydration_levels'].mean() * 0.2)), 2)
}

# Healthy range comparison data
healthy_ranges = {
    'Heart Rate (BPM)': '60-100',
    'Blood Oxygen Level (%)': '95-100',
    'Body Temperature (°C)': '36.1-37.2',
    'Room Temperature (°C)': '18-21',
    'Hydration Levels (%)': '50-60'
}

# Create summary layout
summary_layout = html.Div([
    html.H3("Summary Statistics", style={
        'text-align': 'center',
        'margin-bottom': '20px',
        'font-family': 'Georgia, serif',
        'font-size': '28px',
        'color': '#2C3E50',
        'text-transform': 'uppercase',
        'letter-spacing': '2px',
        'border-bottom': '2px solid #CCC',
        'padding-bottom': '10px'
    }),
    html.Ul([
        html.Li(f"Average Heart Rate (BPM): {summary_stats['Average Heart Rate (BPM)']}", style={'margin-bottom': '10px'}),
        html.Li(f"Average Blood Oxygen Level (%): {summary_stats['Average Blood Oxygen Level (%)']}", style={'margin-bottom': '10px'}),
        html.Li(f"Average Body Temperature (°C): {summary_stats['Average Body Temperature (°C)']}", style={'margin-bottom': '10px'}),
        html.Li(f"Average Room Temperature (°C): {summary_stats['Average Room Temperature (°C)']}", style={'margin-bottom': '10px'}),
        html.Li(f"Average Hydration Levels (%): {summary_stats['Average Hydration Levels (%)']}", style={'margin-bottom': '10px'}),
        html.Li(f"Total Falls Detected: {summary_stats['Total Falls Detected']}", style={'margin-bottom': '10px'}),
        html.Li(f"Potential Epilepsy Events: {summary_stats['Potential Epilepsy Events']}", style={
            'margin-bottom': '10px',
            'color': 'red' if summary_stats['Potential Epilepsy Events'] > 0 else 'inherit',
            'font-weight': 'bold' if summary_stats['Potential Epilepsy Events'] > 0 else 'inherit'
        }),
        html.Li(f"Overall Health Score: {summary_stats['Overall Health Score']}", style={'margin-bottom': '10px'}),
    ], style={'font-size': '18px', 'font-family': 'Arial, sans-serif', 'color': '#34495E'})
], style={
    'border': '1px solid #e7e7e7',
    'border-radius': '10px',
    'padding': '30px',
    'box-shadow': '2px 2px 10px rgba(0,0,0,0.1)',
    'background': 'linear-gradient(135deg, #f7f9fc, #e3e7ed)',
    'width': '80%',
    'margin': '0 auto',
    'text-align': 'left'
})

epilepsy_fig = create_epilepsy_detection_figure(data_cleaned)

def create_epilepsy_events_table(events):
    """Create table of epilepsy events.
    
    Args:
        events: DataFrame containing epilepsy events data
        
    Returns:
        Dash DataTable component
    """
    if events.empty:
        return html.Div("No potential epilepsy events detected.", style={'text-align': 'center', 'padding': '20px'})
    
    # Format the table data
    table_data = []
    for _, row in events.iterrows():
        table_data.append({
            'Time': row['datetime'],
            'Heart Rate': row['heartrate'],
            'Oxygen Level': row['blood_oxygen_level'],
            'Body Temp': row['body_temperature'],
            'HR Variability': row['hrv'] if not pd.isna(row['hrv']) else 'N/A'
        })
    
    # Create the table
    table = dash.dash_table.DataTable(
        id='epilepsy-events-table',
        columns=[{"name": i, "id": i} for i in table_data[0].keys()],
        data=table_data,
        style_table={'overflowX': 'auto'},
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        style_cell={
            'textAlign': 'left',
            'padding': '10px',
            'border': '1px solid lightgrey'
        },
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ]
    )
    
    return table

epilepsy_events_table = create_epilepsy_events_table(epilepsy_events)

epilepsy_explanation = html.Div([
    html.H4("About Epilepsy Detection", style={'text-align': 'center', 'margin-top': '20px'}),
    html.P("""
        Our epilepsy detection algorithm monitors for sudden changes in key health metrics that may indicate a potential 
        epileptic episode. The system flags events when it detects significant simultaneous variations in:
    """),
    html.Ul([
        html.Li("Heart rate (sudden increases or decreases)"),
        html.Li("Blood oxygen levels (rapid drops)"),
        html.Li("Body temperature (unexpected fluctuations)")
    ]),
    html.P("""
        These events are marked with red dots on the graph above. Please note that this is not a diagnostic tool, 
        but rather an alert system to prompt further medical evaluation if multiple events are detected.
    """, style={'font-style': 'italic'})
], style={
    'padding': '20px',
    'background-color': '#f8f9fa',
    'border-radius': '5px',
    'margin-top': '20px'
})

# Create Healthy range comparison layout
healthy_comparison_layout = html.Div([
    html.H3("Comparison with Healthy Standard Ranges (Elderly)", style={
        'text-align': 'center',
        'margin-bottom': '20px',
        'font-family': 'Georgia, serif',
        'font-size': '28px',
        'color': '#2C3E50',
        'text-transform': 'uppercase',
        'letter-spacing': '2px',
        'border-bottom': '2px solid #CCC',
        'padding-bottom': '10px'
    }),
    html.Table([
        html.Tr([html.Th("Metric"), html.Th("Your Average"), html.Th("Healthy Range")]),
        html.Tr([html.Td("Heart Rate (BPM)"), html.Td(f"{summary_stats['Average Heart Rate (BPM)']}"), html.Td(healthy_ranges['Heart Rate (BPM)'])]),
        html.Tr([html.Td("Blood Oxygen Level (%)"), html.Td(f"{summary_stats['Average Blood Oxygen Level (%)']}"), html.Td(healthy_ranges['Blood Oxygen Level (%)'])]),
        html.Tr([html.Td("Body Temperature (°C)"), html.Td(f"{summary_stats['Average Body Temperature (°C)']}"), html.Td(healthy_ranges['Body Temperature (°C)'])]),
        html.Tr([html.Td("Room Temperature (°C)"), html.Td(f"{summary_stats['Average Room Temperature (°C)']}"), html.Td(healthy_ranges['Room Temperature (°C)'])]),
        html.Tr([html.Td("Hydration Levels (%)"), html.Td(f"{summary_stats['Average Hydration Levels (%)']}"), html.Td(healthy_ranges['Hydration Levels (%)'])])
    ], style={'width': '80%', 'margin': '0 auto', 'font-size': '18px', 'border-collapse': 'collapse'})
], style={'width': '80%', 'margin': '0 auto', 'padding': '30px', 'box-shadow': '2px 2px 10px rgba(0,0,0,0.1)', 'border-radius': '10px'})

# Gauge Charts
gauge_fig_heart = go.Figure(go.Indicator(
    mode="gauge+number",
    value=summary_stats['Average Heart Rate (BPM)'],
    title={"text": "Heart Rate (BPM)"},
    gauge={"axis": {"range": [0, 150]},
           "steps": [
               {"range": [60, 100], "color": "lightgreen"},
               {"range": [100, 120], "color": "yellow"},
               {"range": [120, 150], "color": "red"}],
           "bar": {"color": "green"}}
))

gauge_fig_oxygen = go.Figure(go.Indicator(
    mode="gauge+number",
    value=summary_stats['Average Blood Oxygen Level (%)'],
    title={"text": "Blood Oxygen Level (%)"},
    gauge={"axis": {"range": [0, 100]},
           "steps": [
               {"range": [95, 100], "color": "lightgreen"},
               {"range": [90, 95], "color": "yellow"},
               {"range": [0, 90], "color": "red"}],
           "bar": {"color": "green"}}
))

gauge_fig_hydration = go.Figure(go.Indicator(
    mode="gauge+number",
    value=summary_stats['Average Hydration Levels (%)'],
    title={"text": "Hydration Levels (%)"},
    gauge={"axis": {"range": [0, 100]},
           "steps": [
               {"range": [50, 60], "color": "lightgreen"},
               {"range": [40, 50], "color": "yellow"},
               {"range": [0, 40], "color": "red"}],
           "bar": {"color": "green"}}
))

gauge_fig_health_score = go.Figure(go.Indicator(
    mode="gauge+number",
    value=summary_stats['Overall Health Score'],
    title={"text": "Overall Health Score"},
    gauge={"axis": {"range": [0, 100]},
           "steps": [
               {"range": [80, 100], "color": "lightgreen"},
               {"range": [50, 80], "color": "yellow"},
               {"range": [0, 50], "color": "red"}],
           "bar": {"color": "green"}}
))

# Time-Series Analysis
fig_timeseries = go.Figure()
fig_timeseries.add_trace(go.Scatter(
    x=data_cleaned['datetime'], 
    y=data_cleaned['heartrate'], 
    mode='lines', 
    name='Heart Rate (BPM)', 
    line={"color": "#636EFA"}
))
fig_timeseries.add_trace(go.Scatter(
    x=data_cleaned['datetime'], 
    y=data_cleaned['blood_oxygen_level'], 
    mode='lines', 
    name='Blood Oxygen Level (%)', 
    line={"color": "#EF553B"}
))
fig_timeseries.add_trace(go.Scatter(
    x=data_cleaned['datetime'], 
    y=data_cleaned['body_temperature'], 
    mode='lines', 
    name='Body Temperature (°C)', 
    line={"color": "#00CC96"}
))
fig_timeseries.update_layout(
    title="Time-Series Analysis", 
    title_x=0.5, 
    xaxis_title='Time', 
    yaxis_title='Metrics', 
    font={"family": "Arial", "size": 14}, 
    paper_bgcolor='#F9F9F9', 
    plot_bgcolor='#F9F9F9'
)

timeseries_explanation = html.P(
    "This graph shows changes in heart rate, blood oxygen levels, and body temperature over time. "
    "The blue line is for heart rate, the red line shows blood oxygen, and the green line is for body temperature. "
    "You can see how these change throughout the day and week.",
    style={'text-align': 'center', 'margin-top': '20px', 'font-size': '14px'}
)

# Box Plot: Heart Rate by Fall Detection
fig_box = px.box(
    data_cleaned, 
    x='fall_detection', 
    y='heartrate', 
    color='fall_detection', 
    title="Heart Rate by Fall Detection", 
    points="all", 
    template="simple_white"
)
fig_box.update_layout(
    title="Box Plot: Heart Rate by Fall Detection", 
    title_x=0.5, 
    font={"family": "Arial", "size": 14}, 
    paper_bgcolor='#F9F9F9', 
    plot_bgcolor='#F9F9F9'
)

boxplot_explanation = html.P(
    "This box plot shows your heart rate when falls were detected (orange) compared to when no falls were detected (blue). "
    "Higher heart rates in the orange section might indicate stress or physical activity during a fall.",
    style={'text-align': 'center', 'margin-top': '20px', 'font-size': '14px'}
)

# Histogram: HRV Distribution
fig_hrv_histogram = px.histogram(
    data_cleaned, 
    x='hrv', 
    nbins=50, 
    title="Heart Rate Variability (HRV) Distribution", 
    template="simple_white"
)
fig_hrv_histogram.update_layout(
    title="Heart rate variability (HRV) Distribution", 
    title_x=0.5, 
    font={"family": "Arial", "size": 14}, 
    paper_bgcolor='#F9F9F9', 
    plot_bgcolor='#F9F9F9'
)

hrv_explanation = html.P(
    "This histogram shows the distribution of your heart rate variability (HRV), which indicates how much your heart rate changes. "
    "High HRV usually suggests relaxation, while low HRV might indicate stress.",
    style={'text-align': 'center', 'margin-top': '20px', 'font-size': '14px'}
)

# Room and Body Temperature Comparison
fig_temp_comparison = go.Figure()
fig_temp_comparison.add_trace(go.Scatter(
    x=data_cleaned['datetime'], 
    y=data_cleaned['room_temperature'], 
    mode='lines', 
    name='Room Temperature (°C)', 
    line={"color": "#1C6BA0"}
))
fig_temp_comparison.add_trace(go.Scatter(
    x=data_cleaned['datetime'], 
    y=data_cleaned['body_temperature'], 
    mode='lines', 
    name='Body Temperature (°C)', 
    line={"color": "#FFA15A"}
))
fig_temp_comparison.update_layout(
    title="Room Vs Body Temperature", 
    title_x=0.5, 
    font={"family": "Arial", "size": 14}, 
    paper_bgcolor='#F9F9F9', 
    plot_bgcolor='#F9F9F9'
)

room_body_temp_explanation = html.P(
    "This graph compares the temperature of your room (blue) with your body temperature (orange) over time. "
    "It helps you see how changes in the room temperature may affect your comfort.",
    style={'text-align': 'center', 'margin-top': '20px', 'font-size': '14px'}
)

# Correlation Heatmap
heatmap = ff.create_annotated_heatmap(
    z=correlation_matrix.values,
    x=list(correlation_matrix.columns),
    y=list(correlation_matrix.index),
    annotation_text=correlation_matrix.round(2).values,
    colorscale='Blues',
    showscale=True
)
heatmap.update_layout(
    title="Correlation Heatmap", 
    title_x=0.5, 
    font={"family": "Arial", "size": 14}, 
    paper_bgcolor='#F9F9F9', 
    plot_bgcolor='#F9F9F9'
)

heatmap_explanation = html.P(
    "This heatmap shows how different metrics relate to each other. For example, a higher room temperature often correlates with higher body temperature, "
    "while heart rate and oxygen levels show a negative relationship.",
    style={'text-align': 'center', 'margin-top': '20px', 'font-size': '14px'}
)

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout for the dashboard with improved styles
app.layout = html.Div([
    html.H1("Health Monitoring Dashboard", style={
        'text-align': 'center', 
        'font-family': 'Georgia, serif',
        'font-size': '36px', 
        'color': '#34495E',
        'padding-top': '20px',
        'padding-bottom': '20px',
        'letter-spacing': '1.5px',
        'border-bottom': '2px solid #CCC'
    }),
    dcc.Tabs([
        dcc.Tab(label='Summary', children=[summary_layout], style={'padding': '20px'}),
        dcc.Tab(label='Epilepsy Detection', children=[
            html.Div([
                html.H2("Seizure Alert System", style={
                    'text-align': 'center',
                    'color': '#2b5876',
                    'font-family': 'Arial, sans-serif',
                    'margin-bottom': '20px'
                }),
                
                # Simple explanation box
                html.Div([
                    html.H3("How This Works:", style={'color': '#2b5876'}),
                    html.P("This system watches for unusual changes in your:"),
                    html.Ul([
                        html.Li("Heart rate (how fast your heart beats)"),
                        html.Li("Blood oxygen (how much oxygen is in your blood)"),
                        html.Li("Body temperature")
                    ]),
                    html.P("When two or more of these change suddenly at the same time, it may indicate a possible seizure."),
                    html.P(html.B("Important: "), "This is not a doctor. It's just an alert to check with your doctor if you see many warnings.", 
                          style={'color': '#e74c3c', 'font-weight': 'bold'})
                ], style={
                    'background-color': '#f8f9fa',
                    'padding': '20px',
                    'border-radius': '10px',
                    'margin-bottom': '30px',
                    'font-size': '18px'
                }),
                
                # Big, clear alert counter
                html.Div([
                    html.Div([
                        html.H3("Total Possible Seizure Alerts", style={'color': '#e74c3c'}),
                        html.Div(f"{summary_stats['Potential Epilepsy Events']}", style={
                            'font-size': '72px',
                            'font-weight': 'bold',
                            'color': '#e74c3c' if summary_stats['Potential Epilepsy Events'] > 0 else '#27ae60'
                        }),
                        html.P("alerts detected" if summary_stats['Potential Epilepsy Events'] != 1 else "alert detected")
                    ], style={'text-align': 'center'})
                ], style={
                    'background-color': '#ffffff',
                    'border': '3px solid #e74c3c' if summary_stats['Potential Epilepsy Events'] > 0 else '3px solid #27ae60',
                    'border-radius': '10px',
                    'padding': '20px',
                    'margin-bottom': '30px',
                    'max-width': '300px',
                    'margin-left': 'auto',
                    'margin-right': 'auto'
                }),
                
                # Simple graph with clear labels
                dcc.Graph(figure=go.Figure(
                    data=[
                        go.Scatter(
                            x=data_cleaned['datetime'],
                            y=data_cleaned['heartrate'],
                            name='Heart Rate',
                            line=dict(color='blue'),
                            mode='lines'
                        ),
                        go.Scatter(
                            x=epilepsy_events['datetime'],
                            y=epilepsy_events['heartrate'],
                            mode='markers',
                            name='Possible Seizure',
                            marker=dict(
                                color='red',
                                size=12,
                                symbol='x'
                            )
                        )
                    ],
                    layout=go.Layout(
                        title='Your Heart Rate with Possible Seizures Marked',
                        xaxis=dict(title='Time'),
                        yaxis=dict(title='Heart Rate (beats per minute)'),
                        plot_bgcolor='white',
                        paper_bgcolor='white',
                        font=dict(size=16)
                    )
                )),
                
                # When alerts happened
                html.H3("When Alerts Happened:", style={'text-align': 'center', 'margin-top': '30px'}),
                
                html.Div([
                    dash.dash_table.DataTable(
                        id='seizure-events-table',
                        data=epilepsy_events[['datetime', 'heartrate', 'blood_oxygen_level']].to_dict('records') if not epilepsy_events.empty else [],
                        columns=[
                            {'name': 'Time', 'id': 'datetime'},
                            {'name': 'Heart Rate', 'id': 'heartrate'},
                            {'name': 'Oxygen Level', 'id': 'blood_oxygen_level'}
                        ],
                        style_table={'overflowX': 'auto'},
                        style_cell={
                            'textAlign': 'center',
                            'padding': '10px',
                            'fontSize': '16px'
                        },
                        style_header={
                            'backgroundColor': '#2b5876',
                            'color': 'white',
                            'fontWeight': 'bold'
                        },
                        style_data_conditional=[
                            {
                                'if': {'row_index': 'odd'},
                                'backgroundColor': 'rgb(248, 248, 248)'
                            }
                        ]
                    ) if not epilepsy_events.empty else html.Div("No seizure alerts detected - this is good!", style={
                        'text-align': 'center',
                        'font-size': '20px',
                        'color': '#27ae60',
                        'padding': '20px'
                    })
                ], style={'margin-top': '20px'}),
                
                # What to do section
                html.Div([
                    html.H3("What Should I Do?", style={'color': '#2b5876'}),
                    html.Ul([
                        html.Li("If you see red X marks on the graph, make a note of when they happened"),
                        html.Li("Try to remember how you were feeling at those times"),
                        html.Li("Show this information to your doctor at your next visit"),
                        html.Li("If you have many alerts close together, call your doctor sooner")
                    ]),
                    html.P("Remember: This system isn't perfect. Always trust how you feel over what the monitor says.", 
                          style={'font-style': 'italic'})
                ], style={
                    'background-color': '#e3f2fd',
                    'padding': '20px',
                    'border-radius': '10px',
                    'margin-top': '30px',
                    'font-size': '18px'
                })
            ], style={
                'padding': '20px',
                'max-width': '900px',
                'margin': '0 auto'
            })
        ], style={'padding': '20px'}),
        dcc.Tab(label='Comparison with Healthy Ranges', children=[healthy_comparison_layout], style={'padding': '20px'}),
        dcc.Tab(label='Gauge Charts', children=[
            html.H3("GAUGE CHARTS FOR KEY HEALTH METRICS", style={
                'text-align': 'center',
                'margin-bottom': '20px',
                'font-family': 'Georgia, serif',
                'font-size': '28px',
                'color': '#2C3E50',
                'text-transform': 'uppercase',
                'letter-spacing': '2px',
                'border-bottom': '2px solid #CCC',
                'padding-bottom': '10px'
            }),
            html.Div([
                dcc.Graph(figure=gauge_fig_heart),
                dcc.Graph(figure=gauge_fig_oxygen),
                dcc.Graph(figure=gauge_fig_hydration),
                dcc.Graph(figure=gauge_fig_health_score)
            ], style={'display': 'flex', 'flex-direction': 'column', 'align-items': 'center'})
        ], style={'padding': '20px'}),
        dcc.Tab(label='Time-Series Analysis', children=[dcc.Graph(figure=fig_timeseries), timeseries_explanation], style={'padding': '20px'}),
        dcc.Tab(label='Box Plot: Heart Rate by Fall Detection', children=[dcc.Graph(figure=fig_box), boxplot_explanation], style={'padding': '20px'}),
        dcc.Tab(label='HRV Distribution', children=[dcc.Graph(figure=fig_hrv_histogram), hrv_explanation], style={'padding': '20px'}),
        dcc.Tab(label='Room vs Body Temperature', children=[dcc.Graph(figure=fig_temp_comparison), room_body_temp_explanation], style={'padding': '20px'}),
        dcc.Tab(label='Correlation Heatmap', children=[dcc.Graph(figure=heatmap), heatmap_explanation], style={'padding': '20px'}),
        dcc.Tab(label='Respiratory Alerts', children=[
            html.Div([
                html.H3("Blood Oxygen Trend for Respiratory Alerts", style={
                    'text-align': 'center',
                    'margin-bottom': '20px',
                    'font-family': 'Georgia, serif',
                    'font-size': '28px',
                    'color': '#2C3E50',
                    'text-transform': 'uppercase',
                    'letter-spacing': '2px',
                    'border-bottom': '2px solid #CCC',
                    'padding-bottom': '10px'
                }),
                html.Div([
                    html.Div([
                        html.Label("Select Rolling Window Size:", style={'font-weight': 'bold'}),
                        dcc.Slider(
                            id='window-size-slider',
                            min=1,
                            max=15,
                            step=1,
                            value=5,
                            marks={i: str(i) for i in range(1, 16)},
                            tooltip={"placement": "bottom", "always_visible": True}
                        )
                    ], style={'width': '45%', 'display': 'inline-block', 'padding': '20px'}),
                    html.Div([
                        html.Label("Set Alert Threshold (%):", style={'font-weight': 'bold'}),
                        dcc.Slider(
                            id='threshold-slider',
                            min=85,
                            max=95,
                            step=0.5,
                            value=92,
                            marks={i: str(i) for i in range(85, 96)},
                            tooltip={"placement": "bottom", "always_visible": True}
                        )
                    ], style={'width': '45%', 'display': 'inline-block', 'padding': '20px'})
                ], style={'display': 'flex', 'justify-content': 'space-between'}),
                dcc.Graph(id='oxygen-trend-graph'),
                html.Div(id='alert-summary', style={
                    'margin-top': '20px',
                    'padding': '15px',
                    'border-radius': '5px',
                    'background-color': '#f8f9fa'
                }),
                html.Div([
                    html.H4("About Respiratory Alerts", style={'text-align': 'center', 'margin-top': '20px'}),
                    html.P("""
                        This chart monitors your blood oxygen levels over time. The red dashed line shows the alert threshold - 
                        points below this level may indicate potential respiratory issues. The orange dotted line shows the 
                        rolling average, which helps identify trends. Low blood oxygen (SpO2) can be a sign of:
                    """),
                    html.Ul([
                        html.Li("Respiratory conditions like COPD or asthma"),
                        html.Li("Sleep apnea or other sleep-related breathing disorders"),
                        html.Li("Circulatory problems"),
                        html.Li("High altitude effects")
                    ]),
                    html.P("""
                        If you see frequent alerts, please consult with your healthcare provider. Occasional brief dips may 
                        be normal, especially during sleep.
                    """, style={'font-style': 'italic'})
                ], style={
                    'padding': '20px',
                    'background-color': '#f8f9fa',
                    'border-radius': '5px',
                    'margin-top': '20px'
                })
            ], style={'padding': '20px'})
        ], style={'padding': '20px'})
    ], style={'background-color': '#EAEAEA', 'font-family': 'Arial, sans-serif', 'font-size': '16px', 'border': '1px solid #CCC', 'border-radius': '10px'})
], style={'max-width': '1200px', 'margin': '0 auto', 'padding': '20px', 'background-color': '#FFFFFF', 'box-shadow': '0px 4px 10px rgba(0,0,0,0.1)'})

# Callback for Blood Oxygen Trend
@app.callback(
    [Output('oxygen-trend-graph', 'figure'),
     Output('alert-summary', 'children')],
    [Input('window-size-slider', 'value'),
     Input('threshold-slider', 'value')]
)
def update_oxygen_trend(window_size, threshold):
    """Update oxygen trend graph based on slider values.
    
    Args:
        window_size: Size of rolling window for average calculation
        threshold: Oxygen level threshold for alerts
        
    Returns:
        tuple: (figure, alert_summary)
    """
    # Create the figure
    fig = create_oxygen_trend_figure(data_cleaned, window_size, threshold)
    
    # Calculate alert statistics
    alert_points = data_cleaned[data_cleaned['blood_oxygen_level'] < threshold]
    total_alerts = len(alert_points)
    avg_duration = "N/A"
    max_drop = "N/A"
    
    if total_alerts > 0:
        # Calculate average duration of alerts (in minutes)
        alert_groups = (alert_points['datetime'].diff() > timedelta(minutes=5)).cumsum()
        durations = alert_points.groupby(alert_groups)['datetime'].apply(lambda x: (x.max() - x.min()).total_seconds() / 60)
        avg_duration = f"{durations.mean():.1f} minutes"
        
        # Calculate maximum drop below threshold
        max_drop = f"{(threshold - alert_points['blood_oxygen_level'].min()):.1f}%"
    
    # Create alert summary
    alert_summary = html.Div([
        html.H4("Alert Summary", style={'text-align': 'center', 'margin-bottom': '10px'}),
        html.Table([
            html.Tr([html.Td("Total Alert Points:"), html.Td(f"{total_alerts}", style={'font-weight': 'bold' if total_alerts > 0 else 'normal', 'color': 'red' if total_alerts > 0 else 'inherit'})]),
            html.Tr([html.Td("Average Alert Duration:"), html.Td(avg_duration)]),
            html.Tr([html.Td("Maximum Drop Below Threshold:"), html.Td(max_drop)])
        ], style={'width': '50%', 'margin': '0 auto', 'font-size': '16px'})
    ])

    return fig, alert_summary

# Run the app
if __name__ == '__main__':
    app.run(debug=True, port=8052)

## Comprehensive Overview of the Health Monitoring Dashboard Code:

### 1. **Importing Required Libraries:**
   - **dash, dcc (dash_core_components), html (dash_html_components)**: Core libraries from Dash used to build the interactive dashboard for health monitoring.
   - **plotly.express (px) and plotly.graph_objs (go)**: Libraries from Plotly for creating interactive visualizations, including simple (express) and more detailed (graph_objs) plotting tools.
   - **pandas**: A library for data manipulation and analysis, essential for loading, cleaning, and processing health data from the CSV file.
   - **numpy**: A numerical library used to calculate Heart Rate Variability (HRV) by computing differences between heart rate measurements.

### 2. **Loading and Preprocessing the Data:**
   - **Loading the CSV File**: The code loads health monitoring data from a CSV file named `HealthData.csv`.
   - **Datetime Conversion**: The date and timestamp columns are combined into a single `datetime` column to support time-series analysis.
   - **Data Cleaning**: The key health metrics such as heart rate, blood oxygen level, body temperature, room temperature, and hydration levels are converted to numeric format. Any missing values are dropped to ensure data integrity.
   - **HRV Calculation**: Heart Rate Variability (HRV) is computed by calculating the difference between consecutive heart rate values (`data['heartrate'].diff()`).

### 3. **Calculating Key Metrics:**
   - **Summary Statistics**: 
     - Average Heart Rate (BPM)
     - Average Blood Oxygen Level (%)
     - Average Body Temperature (°C)
     - Average Room Temperature (°C)
     - Average Hydration Levels (%)
     - Total Falls Detected
   
   - **Overall Health Score**: This is calculated based on an average of key metrics (heart rate, blood oxygen level, body temperature), providing an overall view of the user’s health on the summary page.

   - **Healthy Standard Ranges**: Predefined healthy ranges for elderly individuals are included to provide a comparison against the user’s actual data.

### 4. **Initializing the Dash App:**
   - `app = dash.Dash(__name__)`: Initializes the Dash app.
   - The `app.layout` defines the structure of the dashboard and how the user interacts with different components (visualizations, tabs).

### 5. **Creating Interactive Visualizations:**
   Each visualization provides an intuitive way for elderly users to monitor and understand their health metrics:

   #### A. **Gauge Charts for Key Health Metrics**:
   - **Heart Rate (BPM)**, **Blood Oxygen Level (%)**, and **Hydration Levels (%)** are represented using gauge charts. These charts display user metrics compared to healthy standard ranges, with color-coded indicators (green = healthy, yellow = caution, red = alert).

   #### B. **Summary Page**:
   - Displays key health statistics like heart rate, blood oxygen levels, body temperature, room temperature, hydration levels, and the total number of falls detected.
   - The overall health score is also displayed, giving users a quick snapshot of their health condition.

   #### C. **Time-Series Analysis**:
   - **Graph Description**: This visualization shows changes in Heart Rate (BPM), Blood Oxygen Level (%), and Body Temperature (°C) over time.
   - **Explanation**: The blue line represents heart rate, the red line shows blood oxygen, and the green line is for body temperature, allowing the user to easily track trends in their health metrics.

   #### D. **Box Plot: Heart Rate by Fall Detection**:
   - **Graph Description**: A box plot comparing heart rate when falls are detected (orange) versus when no falls are detected (blue).
   - **Explanation**: Higher heart rates during falls may indicate physical activity or stress.

   #### E. **HRV Distribution (Heart Rate Variability)**:
   - **Graph Description**: This histogram shows the distribution of Heart Rate Variability (HRV), which measures how much the user’s heart rate changes over time.
   - **Explanation**: Higher HRV typically indicates relaxation, while lower HRV could signal stress or discomfort.

   #### F. **Room and Body Temperature Comparison**:
   - **Graph Description**: This graph shows how the room temperature (blue line) compares to the body temperature (orange line) over time.
   - **Explanation**: It helps users see if fluctuations in room temperature might affect their body temperature and overall comfort.

   #### G. **Correlation Heatmap**:
   - **Graph Description**: A heatmap showing how different health metrics (e.g., heart rate, blood oxygen level, body temperature, hydration levels) correlate with each other.
   - **Explanation**: A positive correlation means two metrics tend to increase together, while a negative correlation means one metric decreases as the other increases.

### 6. **Comparison with Healthy Ranges:**
   - **Page Description**: This page provides a side-by-side comparison between the user’s average metrics and the healthy standard ranges for elderly individuals. It helps users understand how their health metrics compare with optimal values.

### 7. **Creating the Layout**:
   - The layout is defined using `html.Div` to organize the structure of the dashboard. Users can navigate between different tabs using `dcc.Tabs`.
   - Each tab contains a unique visualization (gauge charts, time-series analysis, box plot, HRV distribution, room vs. body temperature comparison, and correlation heatmap).

### 8. **Running the App**:
   - `if __name__ == '__main__':` ensures that the Dash app runs only when the script is executed directly.
   - `app.run_server(debug=True)` starts the web server, allowing users to interact with the dashboard in their browser.

### Summary of Key Features:
   - **Gauge Charts**: Visualize heart rate, blood oxygen level, and hydration against healthy ranges with easy-to-read color codes.
   - **Comparison Page**: Compares the user’s metrics against healthy standard ranges for elderly individuals, providing a clear understanding of their health status.
   - **Time-Series Analysis**: Shows how key metrics change over time, allowing users to track trends.
   - **Box Plot**: Compares heart rate during falls versus non-fall situations.
   - **HRV Histogram**: Displays fluctuations in heart rate, indicating stress or relaxation levels.
   - **Room vs. Body Temperature**: Compares room temperature with body temperature to assess comfort.
   - **Correlation Heatmap**: Helps users understand how different health metrics relate to each other.

This dashboard is designed with elderly users in mind, offering intuitive visualizations and explanations to help them monitor their health and understand key trends.
